In [1]:
# Let's first import required modules for this notebook.
import json
import requests
import sys
# This "sys.path.extend()" statement allows us to move up the directory hierarchy 
# and access the Computer Vision Repository utils_cv package
sys.path.extend([".", "../.."])

# Import AzureML modules that are required for this notebook
from azureml.core import Workspace, Environment
from azureml.core.model import Model, InferenceConfig

# Import custom vision utilities for retrieving workspaces and for testing
from utils_cv.common.azureml import get_or_create_workspace
from utils_cv.common.data import data_path
from utils_cv.common.image import ims2strlist

In [2]:
subscription_id = "0ca618d2-22a8-413a-96d0-0f1b531129c3"
resource_group = "cvbp_project_resources"  
workspace_name = "cvws"  
workspace_region = "eastus"

In [3]:
ws = get_or_create_workspace(
                subscription_id,
                resource_group,
                workspace_name,
                workspace_region)

# Let's print the workspace details
print("Workspace name: " + ws.name,
      "Workspace region: " + ws.location,
      "Subscription ID: " + ws.subscription_id,
      "Resource Group: " + ws.resource_group, sep = "\n")

Workspace name: cvws
Workspace region: eastus
Subscription ID: 0ca618d2-22a8-413a-96d0-0f1b531129c3
Resource Group: cvbp_project_resources


In [4]:
model = Model(ws, "im_similarity_resnet18")

In [5]:
scoring_script = "score.py"

In [ ]:
from utils_cv.classification.model import IMAGENET_IM_SIZE, model_to_learner
from fastai.vision import models, cnn_learner
learn = model_to_learner(models.resnet18(pretrained=True), IMAGENET_IM_SIZE)

In [ ]:
from fastai.vision import cnn_learner

In [ ]:
global learner

In [ ]:
learner = cnn_learner()

In [ ]:
learner

In [ ]:
# Standard python libraries
import sys
import os
import numpy as np
from pathlib import Path
import random
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm
import zipfile
from zipfile import ZipFile

# Fast.ai
import fastai
from fastai.vision import (
    load_learner,
    cnn_learner,
    DatasetType,
    ImageList,
    imagenet_stats,
    models,
    PIL
)

# Computer Vision repository
sys.path.extend([".", "../../.."])  # to access the utils_cv library
from utils_cv.classification.data import Urls
from utils_cv.common.data import unzip_url
from utils_cv.common.gpu import which_processor, db_num_workers
from utils_cv.similarity.metrics import compute_distances
from utils_cv.similarity.model import compute_features_learner
from utils_cv.similarity.plot import plot_distances, plot_ranks_distribution

In [ ]:
# Data location
DATA_PATH = unzip_url(Urls.fridge_objects_path, exist_ok=True)

# Image reader configuration
BATCH_SIZE = 16
IM_SIZE = 300

# Number of comparison of nearest neighbor versus exhaustive search for accuracy computation
NUM_RANK_ITER = 100

# Size of thumbnail images in pixels
MAX_SIZE = (150, 150)

In [ ]:
# Load images into fast.ai's ImageDataBunch object
random.seed(642)
data = (
    ImageList.from_folder(DATA_PATH)
    .split_by_rand_pct(valid_pct=0.8, seed=20)
    .label_from_folder()
    .transform(size=IM_SIZE)
    .databunch(bs=BATCH_SIZE, num_workers = db_num_workers())
    .normalize(imagenet_stats)
)
print(f"Training set: {len(data.train_ds.x)} images, validation set: {len(data.valid_ds.x)} images")

In [ ]:
learner = cnn_learner(data, models.resnet18, ps=0)

In [ ]:
embedding_layer = learner.model[1][-2]

In [ ]:
embedding_layer

In [ ]:
# Load images into fast.ai's ImageDataBunch object
random.seed(642)
data = (
    ImageList.from_folder(DATA_PATH)
    .split_by_rand_pct(valid_pct=0.8, seed=20)
    .label_from_folder()
    .transform(size=IM_SIZE)
    .databunch(bs=BATCH_SIZE, num_workers = db_num_workers())
    .normalize(imagenet_stats)
)
print(f"Training set: {len(data.train_ds.x)} images, validation set: {len(data.valid_ds.x)} images")

In [ ]:
from torch.nn import Module
from torch import Tensor

class SaveFeatures:
    """Hook to save the features in the intermediate layers
    Source: https://forums.fast.ai/t/how-to-find-similar-images-based-on-final-embedding-layer/16903/13
    Args:
        model_layer (nn.Module): Model layer
    """

    features = None

    def __init__(self, model_layer: Module):
        self.hook = model_layer.register_forward_hook(self.hook_fn)
        self.features = None

    def hook_fn(self, module: Module, input: Tensor, output: Tensor):
        out = output.detach().cpu().numpy()
        if isinstance(self.features, type(None)):
            self.features = out
        else:
            self.features = np.row_stack((self.features, out))

    def remove(self):
        self.hook.remove()

In [ ]:
#export model
output_folder = os.path.join(os.getcwd(), 'outputs')
model_name = 'im_similarity_resnet18'  # Name we will give our model both locally and on Azure
pickled_model_name = f'{model_name}.pkl'
os.makedirs(output_folder, exist_ok=True)

learner.export(os.path.join(output_folder, pickled_model_name))

In [ ]:
model = Model.register(
    model_path = os.path.join('outputs', pickled_model_name),
    model_name = model_name,
    tags = {"Model": "Pretrained ResNet18"},
    description = "Image similarity",
    workspace = ws
)

In [ ]:
model = Model(ws, "im_similarity_resnet18")

In [ ]:
output_folder = os.path.join(os.getcwd(), 'outputs')
model_name = 'im_classif_resnet18'  # Name we will give our model both locally and on Azure
pickled_model_name = f'{model_name}.pkl'
os.makedirs(output_folder, exist_ok=True)

learn.export(os.path.join(output_folder, pickled_model_name))

In [6]:
%%writefile $scoring_script
# Copyright (c) Microsoft. All rights reserved.
# Licensed under the MIT license.

import os
import json

from base64 import b64decode
from io import BytesIO

from azureml.core.model import Model
from fastai.vision import load_learner, open_image

# Required for image similarity to save the features in the intermediate layers
from torch.nn import Module
from torch import Tensor
import numpy as np

class SaveFeatures:
    """Hook to save the features in the intermediate layers
    Source: https://forums.fast.ai/t/how-to-find-similar-images-based-on-final-embedding-layer/16903/13
    Args:
        model_layer (nn.Module): Model layer
    """

    features = None

    def __init__(self, model_layer: Module):
        self.hook = model_layer.register_forward_hook(self.hook_fn)
        self.features = None

    def hook_fn(self, module: Module, input: Tensor, output: Tensor):
        out = output.detach().cpu().numpy()
        if isinstance(self.features, type(None)):
            self.features = out
        else:
            self.features = np.row_stack((self.features, out))

    def remove(self):
        self.hook.remove()

def init():
    global learner
    model_path = Model.get_model_path(model_name='im_similarity_resnet18')
    # ! We cannot use the *model_name* variable here otherwise the execution on Azure will fail !

    model_dir_path, model_filename = os.path.split(model_path)
    learner = load_learner(model_dir_path, model_filename)


def run(raw_data):

    # Expects raw_data to be a list within a json file
    result = []    
    
    for im_string in json.loads(raw_data)['data']:
        im_bytes = b64decode(im_string)
        try:
            im = open_image(BytesIO(im_bytes))
            
            # feature extraction
            # use penultimate layer as image representation
            embedding_layer = learner.model[1][-2]
            
            featurizer = SaveFeatures(embedding_layer)
            featurizer.features = None
            
            pred_class, pred_idx, outputs = learner.predict(im)
            
            features = featurizer.features[0][:]
            featurizer.features = None
            
             # "probability": str(outputs[pred_idx].item()),
            result.append({"label": str(pred_class), 
                           "features": "[" + ','.join(map(lambda feature: str(feature), features)) + "]"})
        except Exception as e:
            #, "probability": ''
            result.append({"label": str(e)})
    return result

Overwriting score.py


In [7]:
cv_test_env = Environment.get(workspace=ws, name="im_similarity_resnet18")

In [ ]:


# Create a deployment-specific yaml file from classification/environment.yml
try:
    generate_yaml(
        directory=os.path.join(root_path()), 
        ref_filename='environment.yml',
        needed_libraries=['pytorch', 'spacy', 'fastai', 'dataclasses', 'numpy'],
        conda_filename='myenv.yml'
    )
    # Note: Take a look at the generate_yaml() function for details on how to create your yaml file from scratch

except FileNotFoundError:
    raise FileNotFoundError("The *environment.yml* file is missing - Please make sure to retrieve it from the github repository")



In [ ]:
from azureml.core import Environment
from azureml.core.environment import DEFAULT_CPU_IMAGE

cv_test_env = Environment.from_conda_specification(name="im_similarity_resnet18", file_path="myenv.yml")

# specifying the latest required inferencing stack to be used for deployment
cv_test_env.inferencing_stack_version="latest"

# We will be using the default CPU image for Azure Machine Learning as the base image
# and will add required packages for inferencing
cv_test_env.docker.base_dockerfile="""FROM {}
RUN apt-get update && \
    apt-get install -y libssl-dev build-essential libgl1-mesa-glx
""".format(DEFAULT_CPU_IMAGE)

# setting docker.base_image to None to use the base_dockerfile specified above to build the image
cv_test_env.docker.base_image=None

# Now, let's try registering the environment. 
# You'll be able to see the specified environment printed out.
cv_test_env.register(ws)

In [8]:
inference_config = InferenceConfig(entry_script="score.py", environment=cv_test_env)

inference_image = Model.package(ws, [model], inference_config)
# Setting show_output to True to stream the logs from the Docker image build process
inference_image.wait_for_creation(show_output=True)

2020/05/07 00:57:12 Downloading source code...
2020/05/07 00:57:14 Finished downloading source code
2020/05/07 00:57:14 Creating Docker network: acb_default_network, driver: 'bridge'
2020/05/07 00:57:15 Successfully set up Docker network: acb_default_network
2020/05/07 00:57:15 Setting up Docker configuration...
2020/05/07 00:57:15 Successfully set up Docker configuration
2020/05/07 00:57:15 Logging in to registry: cvwsbbd59dea.azurecr.io
2020/05/07 00:57:16 Successfully logged into cvwsbbd59dea.azurecr.io
2020/05/07 00:57:16 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/05/07 00:57:16 Launching container with name: acb_step_0
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /acb/home/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login S

Successfully built c472d3654c90
Successfully tagged cvwsbbd59dea.azurecr.io/azureml/azureml_4057fba1236d34a7925b0cfac1bd21ba:latest
2020/05/07 00:59:21 Successfully executed container: acb_step_1
2020/05/07 00:59:21 Executing step ID: acb_step_2. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/05/07 00:59:21 Pushing image: cvwsbbd59dea.azurecr.io/azureml/azureml_4057fba1236d34a7925b0cfac1bd21ba:latest, attempt 1
The push refers to repository [cvwsbbd59dea.azurecr.io/azureml/azureml_4057fba1236d34a7925b0cfac1bd21ba]
0c8ae8a87ef0: Preparing
e3f41d412dd1: Preparing
326b0c8bc9ec: Preparing
6868413ff53e: Preparing
bea767d456c6: Preparing
75506b49065c: Preparing
738b9109bef2: Preparing
53bc78b4747b: Preparing
2ed86ec80607: Preparing
91e112657b95: Preparing
0ef3515dfbeb: Preparing
ba4b595b05f2: Preparing
b81662581104: Preparing
8a15ef5cded3: Preparing
36526c4755cf: Preparing
e1171d4d60ca: Preparing
6ef1a8ae63b7: Preparing
85389f9ead9e: Preparing
f2608f66a0e3: Pr

In [9]:
image_location = inference_image.location
print("image location on ACR: {}".format(image_location))

image location on ACR: cvwsbbd59dea.azurecr.io/azureml/azureml_4057fba1236d34a7925b0cfac1bd21ba@sha256:c389d56f44cde1c52566f9fa48e0ef0cfe8e41ebc8ec72146d0a9236bdbf5416


In [ ]:
!az login

In [10]:
appservice_plan_name = "im_similarity_webapp_plan"

In [11]:
!az appservice plan create --resource-group {resource_group} --name {appservice_plan_name} --sku P1v2 --is-linux

{[K - Finished ..
  "freeOfferExpirationTime": null,
  "geoRegion": "Central US",
  "hostingEnvironmentProfile": null,
  "hyperV": false,
  "id": "/subscriptions/0ca618d2-22a8-413a-96d0-0f1b531129c3/resourceGroups/cvbp_project_resources/providers/Microsoft.Web/serverfarms/im_similarity_webapp_plan",
  "isSpot": false,
  "isXenon": false,
  "kind": "linux",
  "location": "Central US",
  "maximumElasticWorkerCount": 1,
  "maximumNumberOfWorkers": 30,
  "name": "im_similarity_webapp_plan",
  "numberOfSites": 0,
  "perSiteScaling": false,
  "provisioningState": "Succeeded",
  "reserved": true,
  "resourceGroup": "cvbp_project_resources",
  "sku": {
    "capabilities": null,
    "capacity": 1,
    "family": "Pv2",
    "locations": null,
    "name": "P1v2",
    "size": "P1v2",
    "skuCapacity": null,
    "tier": "PremiumV2"
  },
  "spotExpirationTime": null,
  "status": "Ready",
  "subscription": "0ca618d2-22a8-413a-96d0-0f1b531129c3",
  "tags": null,
  "targetWorkerCount": 0,
  "targetWork

In [12]:
webapp_name = "im-similarity-resnet18-webapp"

In [13]:
!az webapp create --resource-group {resource_group} --plan {appservice_plan_name} --name {webapp_name} --deployment-container-image-name {image_location}

No credential was provided to access Azure Container Registry. Trying to look up...
{
  "availabilityState": "Normal",
  "clientAffinityEnabled": true,
  "clientCertEnabled": false,
  "clientCertExclusionPaths": null,
  "cloningInfo": null,
  "containerSize": 0,
  "dailyMemoryTimeQuota": 0,
  "defaultHostName": "im-similarity-resnet18-webapp.azurewebsites.net",
  "enabled": true,
  "enabledHostNames": [
    "im-similarity-resnet18-webapp.azurewebsites.net",
    "im-similarity-resnet18-webapp.scm.azurewebsites.net"
  ],
  "ftpPublishingUrl": "ftp://waws-prod-dm1-171.ftp.azurewebsites.windows.net/site/wwwroot",
  "hostNameSslStates": [
    {
      "hostType": "Standard",
      "ipBasedSslResult": null,
      "ipBasedSslState": "NotConfigured",
      "name": "im-similarity-resnet18-webapp.azurewebsites.net",
      "sslState": "Disabled",
      "thumbprint": null,
      "toUpdate": null,
      "toUpdateIpBasedSsl": null,
      "virtualIp": null
    },
    {
      "hostType": "Repository",


In [14]:
acr_name = image_location.split(".")[0]
print("acr_name: {}".format(acr_name))

acr_name: cvwsbbd59dea


In [ ]:
!az acr credential show --name {acr_name}

In [15]:
acr_username = "cvwsbbd59dea"
acr_password = "lMouGaj0np+FsKVuIgBUzADfuXHRxppL"

In [16]:
acr_server_url = image_location.split("/")[0]
print("acr_server_url: {}".format(acr_server_url))

acr_server_url: cvwsbbd59dea.azurecr.io


In [18]:
!az webapp config container set --resource-group {resource_group} --name {webapp_name} --docker-custom-image-name {image_location} --docker-registry-server-url {acr_server_url} --docker-registry-server-user {acr_username} --docker-registry-server-password {acr_password}

[
  {
    "name": "WEBSITES_ENABLE_APP_SERVICE_STORAGE",
    "slotSetting": false,
    "value": "false"
  },
  {
    "name": "DOCKER_REGISTRY_SERVER_URL",
    "slotSetting": false,
    "value": "cvwsbbd59dea.azurecr.io"
  },
  {
    "name": "DOCKER_REGISTRY_SERVER_USERNAME",
    "slotSetting": false,
    "value": "cvwsbbd59dea"
  },
  {
    "name": "DOCKER_REGISTRY_SERVER_PASSWORD",
    "slotSetting": false,
    "value": null
  },
  {
    "name": "DOCKER_CUSTOM_IMAGE_NAME",
    "value": "DOCKER|cvwsbbd59dea.azurecr.io/azureml/azureml_4057fba1236d34a7925b0cfac1bd21ba@sha256:c389d56f44cde1c52566f9fa48e0ef0cfe8e41ebc8ec72146d0a9236bdbf5416"
  }
]


In [19]:
!az webapp log config --resource-group {resource_group} --name {webapp_name} --docker-container-logging filesystem

{
  "applicationLogs": {
    "azureBlobStorage": {
      "level": "Off",
      "retentionInDays": null,
      "sasUrl": null
    },
    "azureTableStorage": {
      "level": "Off",
      "sasUrl": null
    },
    "fileSystem": {
      "level": "Off"
    }
  },
  "detailedErrorMessages": {
    "enabled": false
  },
  "failedRequestsTracing": {
    "enabled": false
  },
  "httpLogs": {
    "azureBlobStorage": {
      "enabled": false,
      "retentionInDays": 3,
      "sasUrl": null
    },
    "fileSystem": {
      "enabled": true,
      "retentionInDays": 3,
      "retentionInMb": 100
    }
  },
  "id": "/subscriptions/0ca618d2-22a8-413a-96d0-0f1b531129c3/resourceGroups/cvbp_project_resources/providers/Microsoft.Web/sites/im-similarity-resnet18-webapp/config/logs",
  "kind": null,
  "location": "Central US",
  "name": "logs",
  "resourceGroup": "cvbp_project_resources",
  "type": "Microsoft.Web/sites/config"
}


In [ ]:
!az webapp log config --resource-group {resource_group} --name {webapp_name} --web-server-logging filesystem

In [ ]:
import os
test_image_directory = "https://cvbp.blob.core.windows.net/public/images/"
test_image_filenames = ["cvbp_milk_bottle.jpg", "cvbp_water_bottle.jpg"]
local_test_image_paths = []

for test_image_filename in test_image_filenames:
    req = requests.get(os.path.join(test_image_directory, test_image_filename))
    local_test_image_path = os.path.join(data_path(), test_image_filename)

    with open(local_test_image_path, "wb") as file:
        file.write(req.content)
        local_test_image_paths.append(local_test_image_path)

# Use the utility function im2strlist to get a list containing base64-encoded images decoded into strings
decoded_b64_test_images = ims2strlist(local_test_image_paths)

In [ ]:
scoring_uri = "https://{}.azurewebsites.net/score".format(webapp_name)

headers = {"Content-Type" : "application/json"}

test_image_data = json.dumps({"data" : decoded_b64_test_images})

response = requests.post(scoring_uri, data=test_image_data, headers=headers)

print("Predictions: {}".format(response.json()),
      "Received the scored result in: {}".format(response.elapsed),
      "Response status code: {}".format(response.status_code), sep="\n")

In [ ]:
!az webapp log tail --resource-group $resource_group --name $webapp_name